In [2]:
from dataset import AirQualityDataset
from dataset.raw_data import air_quality_test_data, air_quality_train_data
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

/home/hoang/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dts = AirQualityDataset(
        "../data",
        output_frame_size=24,
        normalize_mean={"humidity": 74.91, "temperature": 26.52, "PM2.5": 43.87},
        normalize_std={"humidity": 15.38, "temperature": 5.77, "PM2.5": 40.12},
        stride=7,
        data_set="train",
        fillnan_fn=lambda x: x.interpolate(option="spline").bfill()
    )

In [12]:
for i in range(len(dts)):
    if dts[i]["features"].size(1) == 0:
        print(i)
        break

5144


In [7]:
dts[0]["features"].shape

torch.Size([8, 168, 3])

In [4]:
raw = air_quality_train_data("../data/data-train/")

In [5]:
from statistics import mean
m, s = [], []

for name, station in raw["input"].items():
    m.append(station["data"]["humidity"].dropna().mean())
    s.append(station["data"]["humidity"].dropna().std())

{
    "mean": mean(m),
    "std": mean(s)
}

{'mean': 75.83080892930954, 'std': 14.527020139735658}